##基礎演習S 後半第3回

NE21-1228F
鈴木　拓己

In [98]:
if(!require("magrittr")){install.packages("magrittr")}; library(magrittr)
if(!require("rvest")){install.packages("rvest")}; library(rvest)
if(!require("stringr")){install.packages("stringr")}; library(stringr)
if(!require("leaflet")){install.packages("leaflet")}; library(leaflet)
if(!require("htmlwidgets")){install.packages("htmlwidgets")}; library(htmlwidgets)
if(!require("maptools")){install.packages("maptools")}; library(maptools)
if(!require("classInt")){install.packages("classInt")}; library(classInt)
if(!require("RColorBrewer")){install.packages("RColorBrewer")}; library(RColorBrewer)
if(!require("rgdal")){install.packages("rgdal")}; library(rgdal)

### Ex.3-4　位置の特定と視覚化
##### 【問題】runmeterというランニングサポートのサービスがある．そのサービスサイトから利用者の位置情報を取得し，地図に表示したい．ここでは，以下のURLにアクセスして一度に取得できるだけのデータを視覚化することにする．
https://twitter.com/search?f=realtime&q=%23runmeter%20lang%3Aja&src=typd

* 同ページを開いて「https://runmeter.com」 で始まるリンクを辿ると位置情報のあるページが現れる．
* リンク先を「ページのソースを表示」で見ると，"maxLatitude", "minLatitude", "maxLongitude", "minLongitude"（最大緯度，最小緯度，最大経度，最小経度）の情報が取得できることが分かる ．
* そこで，これら４つの値を抽出し，緯度の平均と経度の平均から決まる大まかな位置（一地点）をマーカーで地図上に示しなさい．※これを，サイトにアクセスして一度に取得できるデータの数だけ繰り返して表示すること．
* なお，マーカーのラベルは「(経度, 緯度)」とし，ポップアップには，位置情報のあるページへのリンクを表示し，クリックするとリンク先に飛べるようにすること．


In [99]:
library(rvest)
library(magrittr)

url <- "/content/Twitter.html"

dat <- read_html(url)

dat

XPath1 <- "//a[@class='css-4rbku5 css-18t94o4 css-901oao css-16my406 r-1cvl2hr r-1loqt21 r-poiln3 r-bcqeeo r-qvutc0']"	
subj <- html_nodes(dat,xpath=XPath1)
subj

spage <- html_attr(subj,"href")
spage

{html_document}
<html dir="ltr" lang="ja" style="overflow-y: scroll; overscroll-behavior-y: none; font-size: 15px;">
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <body style="background-color: rgb(0, 0, 0);">\n  <noscript>\n    <style> ...

{xml_nodeset (3)}
[1] <a dir="ltr" href="https://t.co/rxUmnEMfN8" rel="noopener noreferrer nofo ...
[2] <a dir="ltr" href="https://t.co/rxUmnEMfN8" rel="noopener noreferrer nofo ...
[3] <a dir="ltr" href="https://t.co/2zDKveQTV4" rel="noopener noreferrer nofo ...

[1] "https://t.co/rxUmnEMfN8" "https://t.co/rxUmnEMfN8"
[3] "https://t.co/2zDKveQTV4"

In [ ]:
for(i in 1;length(spage)){
  url <- spage[i]
  dat <- read_html(url)
  XPath1 <- "//script"	
  subj <- html_nodes(dat,xpath=XPath1)
  text <- html_text(subj);
  sub("\"maxLatitude\":(.*),\"minLatitude\":(.*),\"maxLongitude\":(.*),\"minLongitude\":(.*)","\\1",text,perl=TRUE)
}

In [102]:
#座標の抜き出しができませんでした。そのため手入力で座標を入れ、urlをスクレイピングで出力します。
lng <- c((135.49376601365907+135.4823947885404)/2,(135.49376601365907+135.4823947885404)/2,(140.36573551521724+140.35709448045068)/2)
lat <- c((34.720488055641503+34.670293112317722)/2,(34.720488055641503+34.670293112317722)/2,(37.382286187551216+37.377047465872849)/2)

In [103]:
pop <- paste0("<a href=\"",spage,"\">",spage,"</a>")

label <- paste(lng, lat, sep=", ")    # ラベル

df <- data.frame(lng, lat, pop)
m <- df %>%
    leaflet %>%
    addTiles %>%
    addMarkers(lng=lng, lat=lat, popup=pop, label=label)

saveWidget(m, "twitter_runmeter.html")